In [ ]:
#Random  forest 

import sqlite3
import pandas as pd

In [2]:

from platform import python_version

print(python_version())

3.7.7

In [3]:

from pathlib import Path
db_dir = str(Path.home()) + '/sqlitedb/'
classifier_file = db_dir + 'TwitterUserClassifier.db'
friend_file = db_dir + 'TwitterFriends.db'
conn = sqlite3.connect(classifier_file)
conn.execute('ATTACH DATABASE "' + friend_file + '" AS friends')

Out[3]:

<sqlite3.Cursor at 0x12199dbb20>

In [4]:

#dfUserClass1 =  pd.read_sql_query('select id, 1 class from user_class1 order by random();', conn)
#dfUserClass0 =  pd.read_sql_query('select id, 0 class from user_class0 order by random();', conn)
dfUserClass1 =  pd.read_sql_query('select distinct u.id, 1 class '
                                  'from user_class1 u '
                                  'join friends.user_friend uf '
                                  'on u.id=uf.user_id and uf.friend_id is not null '
                                  'order by random();',
                                  conn)
dfUserClass0 =  pd.read_sql_query('select distinct u.id, 0 class '
                                  'from user_class0 u '
                                  'join friends.user_friend uf '
                                  'on u.id=uf.user_id and uf.friend_id is not null '
                                  'order by random();', conn)
print("User class1 shape: ")
print(dfUserClass1.shape)
print("User class0 shape: ")
print(dfUserClass0.shape)

User class1 shape: 
(3038, 2)
User class0 shape: 
(5261, 2)

In [5]:

dfUserClass1 = dfUserClass1.head(3000)
dfUserClass0 = dfUserClass0.head(5000)
dfUserClassAll = pd.concat([dfUserClass1, dfUserClass0])
dfUserClassAll = dfUserClassAll.sample(frac=1)

In [6]:

splitPos = int(dfUserClassAll.shape[0]*0.70)
dfTrain = dfUserClassAll.iloc[0:splitPos]
dfTest = dfUserClassAll.iloc[splitPos:]
print("User class all shape: ")
print(dfUserClassAll.shape)
print("Data Set Train shape: ")
print(dfTrain.shape)
print("Data Set Test shape: ")
print(dfTest.shape)

User class all shape: 
(8000, 2)
Data Set Train shape: 
(5600, 2)
Data Set Test shape: 
(2400, 2)

In [7]:

dfUserFriend=pd.read_sql_query('select user_id, friend_id '
                               'from friends.user_friend '
                               'where friend_id is not null;', conn)
dfUserFriend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3417929 entries, 0 to 3417928
Data columns (total 2 columns):
 #   Column     Dtype
---  ------     -----
 0   user_id    int64
 1   friend_id  int64
dtypes: int64(2)
memory usage: 52.2 MB

In [8]:

dfAllUsers = pd.read_sql_query('select u.id, u.screen_name, u.name, u.followers_count '
                           'from friends.user u;', conn)
dfAllUsers.set_index(['id'])
dfAllUsers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709151 entries, 0 to 709150
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               709151 non-null  int64 
 1   screen_name      709151 non-null  object
 2   name             709151 non-null  object
 3   followers_count  709151 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 21.6+ MB

In [9]:

def buildTopUser(_df_user_friend, _df_user, _numTopUsers):
    _i1 = _df_user_friend.set_index(['user_id']).index
    _i2 = _df_user.set_index(['id']).index
    _dfTopUser = _df_user_friend[_i1.isin(_i2)]
    _dfTopUser = _dfTopUser.groupby(['friend_id'])['user_id'].count().reset_index(name='count').sort_values(['count'], ascending=False)[:_numTopUsers]
    #_dfTopUser = pd.merge(_dfTopUser, _df_all_users, left_on='friend_id', right_on='id')
    return _dfTopUser

In [10]:

def buildFeatureVectors(_df_user_friend, _df_user, _df_top_user):
    _i1 = _df_user_friend.set_index('user_id').index
    _i2 = _df_user.set_index('id').index
    _dfUserFriend2 = _df_user_friend[_i1.isin(_i2)]
    _i3 = _dfUserFriend2.set_index('friend_id').index
    _i4 = _df_top_user.set_index('friend_id').index
    _dfUserFriend3 = _dfUserFriend2[_i3.isin(_i4)]
    
    _feature_vectors = pd.crosstab(index=_dfUserFriend3['user_id'], columns=_dfUserFriend3['friend_id'], dropna=False)
    _feature_vectors = pd.merge(_feature_vectors, _df_user, left_on='user_id', right_on='id')
    return _feature_vectors

In [11]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score

In [12]:

numTopUser = 500
dfTopUser = buildTopUser(dfUserFriend, dfTrain, numTopUser)
numTopUser=dfTopUser.shape[0]
feature_vectors_train = buildFeatureVectors(dfUserFriend, dfTrain, dfTopUser)
feature_vectors_test = buildFeatureVectors(dfUserFriend, dfTest, dfTopUser)
#Rearrange columns on the test
feature_vectors_test = feature_vectors_test[feature_vectors_train.columns]
X_train = feature_vectors_train.iloc[:, :-2].values
y_train = feature_vectors_train.iloc[:, (numTopUser+1)].values
X_test = feature_vectors_test.iloc[:, :-2].values
y_test = feature_vectors_test.iloc[:, (numTopUser+1)].values

In [13]:

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

Out[13]:

RandomForestClassifier()

In [14]:

y_pred = classifier.predict(X_test)
y_prob = classifier.predict_proba(X_test)[:, 1]
print("ROC AUC: ", roc_auc_score(y_test, y_prob))
print("Classification Report:",)
print (classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test,y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

ROC AUC:  0.9346447723041871
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1395
           1       0.92      0.79      0.85       903

    accuracy                           0.89      2298
   macro avg       0.90      0.87      0.88      2298
weighted avg       0.89      0.89      0.89      2298

Accuracy: 0.8920800696257616
Confusion Matrix:
[[1336   59]
 [ 189  714]]

In [15]:

fi = pd.DataFrame({'feature': list(feature_vectors_train.columns[:-2]),
                   'importance': classifier.feature_importances_}).\
                    sort_values('importance', ascending = False)
fi

Out[15]:
	feature 	importance
496 	1301760719906512896 	0.066097
202 	199992361 	0.051649
142 	92531902 	0.040005
69 	50916567 	0.039931
487 	1269106344528736256 	0.028700
... 	... 	...
278 	953670151 	0.000104
125 	78871747 	0.000100
348 	3968042414 	0.000100
206 	213612447 	0.000078
270 	830434249 	0.000030

500 rows × 2 columns
In [18]:

%matplotlib inline
fi[:40].plot.bar(x='feature', y='importance', figsize=(14,7))

Out[18]:

<matplotlib.axes._subplots.AxesSubplot at 0x1226b1f110>



